In [17]:
# set up the environment
%pylab
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import seaborn as sns
sns.set()
import pandas as pd
import scipy as sp
import scipy.stats
import statsmodels.stats.api as sms
import statsmodels.api as sm
import statsmodels.formula.api as smf

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


/Users/kallus/Applications/anaconda2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [18]:
## veteran earnings
with open("ssaone.sas7bdat") as f:
    veteran = pd.read_sas(f,format="sas7bdat")
veteran = veteran[
        np.isfinite(veteran['EARNVAR']) & np.isfinite(veteran['STDVAR'])
      & np.isfinite(veteran['SAMCOUNT']) & np.isfinite(veteran['POPCOUNT'])
      & (veteran['YEAR']>=88) & (veteran['YEAR']<=91)
      & (veteran['TRANSYY']>=79) & (veteran['TRANSYY']<=82) 
      & (veteran['EDGRP']!=10) 
      & ((veteran['AFQTGRP']<=6) & (veteran['AFQTGRP']>=2))
    ]
veteran.head()

,AFQTGRP,DNWHITE,DOBYY,DQUAL,DVET,EDGRP,TRANSYY,DROP,AGE,PROBVET,...,LAGERN80,LAGERN81,LAGZER76,LAGZER77,LAGZER78,LAGZER79,LAGZER80,LAGZER81,YEAR,CPI
80832,2.0,0.0,57.0,0.0,0.0,3.0,79.0,1.0,1.0,0.196721,...,37.317028,40.202533,7.0,5.0,4.0,4.0,11.0,12.0,88.0,0.868576
80833,2.0,0.0,57.0,0.0,1.0,3.0,79.0,0.0,1.0,0.196721,...,49.391181,45.420181,7.0,3.0,5.0,NaN,NaN,4.0,88.0,0.868576
80834,2.0,0.0,57.0,0.0,0.0,5.0,79.0,5.0,1.0,0.187500,...,53.090354,50.965258,8.0,7.0,NaN,NaN,4.0,5.0,88.0,0.868576
80835,2.0,0.0,57.0,0.0,0.0,6.0,79.0,2.0,1.0,0.341463,...,53.095672,52.262254,7.0,6.0,NaN,NaN,3.0,4.0,88.0,0.868576
80836,2.0,0.0,57.0,0.0,1.0,6.0,79.0,0.0,1.0,0.341463,...,65.766181,66.913091,5.0,NaN,3.0,NaN,NaN,NaN,88.0,0.868576


In [19]:
veteran_white = veteran[veteran['DNWHITE']==0]

In [20]:
smf.wls('EARNVAR ~ DVET', data=veteran_white, weights = veteran_white['POPCOUNT']).fit().summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                EARNVAR   R-squared:                       0.047
Model:                            WLS   Adj. R-squared:                  0.047
Method:                 Least Squares   F-statistic:                     199.2
Date:                Mon, 12 Mar 2018   Prob (F-statistic):           3.43e-44
Time:                        16:24:47   Log-Likelihood:                -39614.
No. Observations:                4048   AIC:                         7.923e+04
Df Residuals:                    4046   BIC:                         7.924e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.391e+04     62.075    224.152      0.000    1.38e+04     1.4e+04
DVET        1233.4368     87.391     14.114      0.000    1062.103    1404.771
==============================================================================
Omnibus:                      595.323   Durbin-Watson:                   1.108
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1604.759
Skew:                           0.800   Prob(JB):                         0.00
Kurtosis:                       5.637   Cond. No.                         2.63
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
smf.wls('EARNVAR ~ DVET + C(AFQTGRP) + DOBYY + C(EDGRP) + LAGERN76 + LAGERN77 + LAGERN78', data=veteran_white, weights = veteran_white['POPCOUNT']).fit().summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                EARNVAR   R-squared:                       0.803
Model:                            WLS   Adj. R-squared:                  0.802
Method:                 Least Squares   F-statistic:                     951.4
Date:                Mon, 12 Mar 2018   Prob (F-statistic):               0.00
Time:                        16:27:24   Log-Likelihood:                -27457.
No. Observations:                3052   AIC:                         5.494e+04
Df Residuals:                    3038   BIC:                         5.503e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept         -2500.7050   3042.211     -0.822      0.411   -8465.706    3464.296
C(AFQTGRP)[T.3.0]  1226.2203    106.997     11.460      0.000    1016.427    1436.013
C(AFQTGRP)[T.4.0]  2330.8242     96.544     24.143      0.000    2141.527    2520.122
C(AFQTGRP)[T.5.0]  3269.0354     93.445     34.984      0.000    3085.814    3452.257
C(AFQTGRP)[T.6.0]  4390.8748     97.902     44.850      0.000    4198.913    4582.836
C(EDGRP)[T.5.0]    3566.9284     65.338     54.592      0.000    3438.817    3695.039
C(EDGRP)[T.6.0]    4806.6978     63.824     75.312      0.000    4681.556    4931.839
C(EDGRP)[T.7.0]    6230.1545    207.851     29.974      0.000    5822.612    6637.697
C(EDGRP)[T.13.0]    616.1375    127.275      4.841      0.000     366.584     865.691
DVET                -63.4675     48.724     -1.303      0.193    -159.004      32.069
DOBYY               176.3570     48.701      3.621      0.000      80.866     271.848
LAGERN76            -22.0893     17.123     -1.290      0.197     -55.663      11.485
LAGERN77            -54.7522     15.019     -3.645      0.000     -84.201     -25.303
LAGERN78             52.5230      6.790      7.735      0.000      39.209      65.837
==============================================================================
Omnibus:                      243.355   Durbin-Watson:                   1.473
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              940.915
Skew:                           0.317   Prob(JB):                    4.82e-205
Kurtosis:                       5.645   Cond. No.                     8.62e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.62e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [22]:
## heterogeneous effects
smf.wls('EARNVAR ~ DNWHITE*DVET + C(AFQTGRP) + DOBYY + C(EDGRP) + LAGERN76 + LAGERN77 + LAGERN78', data=veteran, weights = veteran['POPCOUNT']).fit().summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                EARNVAR   R-squared:                       0.853
Model:                            WLS   Adj. R-squared:                  0.853
Method:                 Least Squares   F-statistic:                     2375.
Date:                Mon, 12 Mar 2018   Prob (F-statistic):               0.00
Time:                        16:34:55   Log-Likelihood:                -55058.
No. Observations:                6152   AIC:                         1.101e+05
Df Residuals:                    6136   BIC:                         1.103e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept         -6919.4414   1744.846     -3.966      0.000   -1.03e+04   -3498.932
C(AFQTGRP)[T.3.0]   928.2536     59.470     15.609      0.000     811.671    1044.837
C(AFQTGRP)[T.4.0]  1836.4871     55.545     33.063      0.000    1727.598    1945.376
C(AFQTGRP)[T.5.0]  2789.1621     55.088     50.631      0.000    2681.170    2897.154
C(AFQTGRP)[T.6.0]  3912.1740     60.295     64.884      0.000    3793.974    4030.374
C(EDGRP)[T.5.0]    3426.6890     45.253     75.724      0.000    3337.978    3515.400
C(EDGRP)[T.6.0]    4808.6430     44.276    108.607      0.000    4721.847    4895.439
C(EDGRP)[T.7.0]    6675.7306    121.520     54.935      0.000    6437.508    6913.953
C(EDGRP)[T.13.0]    750.3555     91.193      8.228      0.000     571.586     929.125
DNWHITE           -2914.6813     49.532    -58.844      0.000   -3011.782   -2817.581
DVET                  1.9213     39.555      0.049      0.961     -75.621      79.463
DNWHITE:DVET        925.6805     65.650     14.100      0.000     796.983    1054.378
DOBYY               254.0313     27.844      9.123      0.000     199.447     308.615
LAGERN76            -12.2322     12.253     -0.998      0.318     -36.253      11.789
LAGERN77            -58.5871     10.767     -5.441      0.000     -79.695     -37.479
LAGERN78             56.5259      4.800     11.777      0.000      47.117      65.935
==============================================================================
Omnibus:                      676.063   Durbin-Watson:                   1.518
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4323.337
Skew:                           0.321   Prob(JB):                         0.00
Kurtosis:                       7.056   Cond. No.                     7.30e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.3e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [23]:
smf.wls('EARNVAR ~ DNWHITE*(DVET + C(AFQTGRP) + DOBYY + C(EDGRP) + LAGERN76 + LAGERN77 + LAGERN78)', data=veteran, weights = veteran['POPCOUNT']).fit().summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                EARNVAR   R-squared:                       0.857
Model:                            WLS   Adj. R-squared:                  0.857
Method:                 Least Squares   F-statistic:                     1364.
Date:                Mon, 12 Mar 2018   Prob (F-statistic):               0.00
Time:                        16:37:10   Log-Likelihood:                -54966.
No. Observations:                6152   AIC:                         1.100e+05
Df Residuals:                    6124   BIC:                         1.102e+05
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                 -2500.7050   2486.395     -1.006      0.315   -7374.912    2373.502
C(AFQTGRP)[T.3.0]          1226.2203     87.448     14.022      0.000    1054.791    1397.649
C(AFQTGRP)[T.4.0]          2330.8242     78.905     29.540      0.000    2176.143    2485.506
C(AFQTGRP)[T.5.0]          3269.0354     76.372     42.804      0.000    3119.319    3418.752
C(AFQTGRP)[T.6.0]          4390.8748     80.015     54.875      0.000    4234.017    4547.733
C(EDGRP)[T.5.0]            3566.9284     53.401     66.796      0.000    3462.244    3671.612
C(EDGRP)[T.6.0]            4806.6978     52.163     92.148      0.000    4704.440    4908.955
C(EDGRP)[T.7.0]            6230.1545    169.876     36.675      0.000    5897.138    6563.171
C(EDGRP)[T.13.0]            616.1375    104.021      5.923      0.000     412.219     820.056
DNWHITE                   -1.177e+04   3880.701     -3.033      0.002   -1.94e+04   -4162.355
DNWHITE:C(AFQTGRP)[T.3.0]  -515.1162    118.072     -4.363      0.000    -746.580    -283.653
DNWHITE:C(AFQTGRP)[T.4.0]  -974.3560    110.739     -8.799      0.000   -1191.443    -757.269
DNWHITE:C(AFQTGRP)[T.5.0]  -999.6621    112.494     -8.886      0.000   -1220.190    -779.134
DNWHITE:C(AFQTGRP)[T.6.0] -1074.2049    134.282     -8.000      0.000   -1337.445    -810.965
DNWHITE:C(EDGRP)[T.5.0]    -541.5898     99.490     -5.444      0.000    -736.626    -346.554
DNWHITE:C(EDGRP)[T.6.0]    -119.2518     97.429     -1.224      0.221    -310.247      71.743
DNWHITE:C(EDGRP)[T.7.0]     767.7380    243.068      3.159      0.002     291.238    1244.238
DNWHITE:C(EDGRP)[T.13.0]    441.9821    208.273      2.122      0.034      33.693     850.271
DVET                        -63.4675     39.822     -1.594      0.111    -141.533      14.598
DOBYY                       176.3570     39.804      4.431      0.000      98.328     254.386
LAGERN76                    -22.0893     13.995     -1.578      0.115     -49.524       5.345
LAGERN77                    -54.7522     12.275     -4.460      0.000     -78.816     -30.688
LAGERN78                     52.5230      5.550      9.464      0.000      41.644      63.402
DNWHITE:DVET               1176.6558     69.920     16.829      0.000    1039.588    1313.723
DNWHITE:DOBYY               156.6141     62.028      2.525      0.012      35.017     278.211
DNWHITE:LAGERN76             56.6372     28.458      1.990      0.047       0.850     112.424
DNWHITE:LAGERN77            -11.1487     24.796     -0.450      0.653     -59.758      37.460
DNWHITE:LAGERN78             -1.6084     11.838     -0.136      0.892     -24.814      21.598
==============================================================================
Omnibus:                      582.183   Durbin-Watson:                   1.548
Prob(Omnibu